## Run all the data analysis notebooks in sequence, and store results and plots in the ./results/ subdirectory.

### Note, the orbit fitting stage (notebooks #8 and #9) are relatively slow due to the large number of MCMC samples required. Because the orbit fits are repeated for different combinations of epochs, expect the overall sequence of notebooks to take ~1 hour to complete.

- 01-centered_HLC_PSF.ipynb - prepare PSF model  
- 02-HLC_RDI.ipynb - Reference differential imaging  
- 03-HLC_photometry_astrometry_ep1.ipynb - Photometry and astrometry, epoch 1
- 04-HLC_photometry_astrometry_ep2.ipynb - Photometry and astrometry, epoch 2
- 05-HLC_photometry_astrometry_ep3.ipynb - Photometry and astrometry, epoch 3
- 06-HLC_photometry_astrometry_ep4.ipynb - Photometry and astrometry, epoch 4
- 07-starshade_photometry_astrometry_ep5.ipynb - Photometry and astrometry, epoch 5
- 08-radial_velocities.ipynb - Radial velocity analysis
- 09-orbit.ipynb - Orbit fit analysis, repeated for different numbers of cumulative epochs
- 10-phase_curve.ipynb - Photometry and phase curve analysis, albedo inference

The nbconvert commands were copied from the example at https://nbconvert.readthedocs.io/en/latest/execute_api.html 

The nbparameterise commands copied from 
https://github.com/takluyver/nbparameterise/blob/master/README.rst

In [1]:
import os
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor
from nbparameterise import (
    extract_parameters, replace_definitions, parameter_values
)

### Reference PSF subtraction (RDI), photometry, and astrometry

In [2]:
nb_run_list = [
        '01-centered_HLC_PSF.ipynb',
        '02-HLC_RDI.ipynb',
        '03-HLC_photometry_astrometry_ep1.ipynb',
        '04-HLC_photometry_astrometry_ep2.ipynb',
        '05-HLC_photometry_astrometry_ep3.ipynb',
        '06-HLC_photometry_astrometry_ep4.ipynb',
        '07-starshade_photometry_astrometry_ep5.ipynb'
]

nb_dir = os.getcwd()

In [3]:
ep = ExecutePreprocessor(timeout=1200)

for nb_fname in nb_run_list:
    full_nb_fname = os.path.join(nb_dir, nb_fname)
    
    with open(nb_fname) as f:
        nb = nbformat.read(f, as_version=4)
    
    _ = ep.preprocess(nb, {'metadata': {'path': nb_dir}})
    
    with open(nb_fname, 'w', encoding='utf-8') as f:
        nbformat.write(nb, f)
        
    print("Finished running {:s}".format(nb_fname))

Finished running 01-centered_HLC_PSF.ipynb
Finished running 02-HLC_RDI.ipynb
Finished running 03-HLC_photometry_astrometry_ep1.ipynb
Finished running 04-HLC_photometry_astrometry_ep2.ipynb
Finished running 05-HLC_photometry_astrometry_ep3.ipynb
Finished running 06-HLC_photometry_astrometry_ep4.ipynb
Finished running 07-starshade_photometry_astrometry_ep5.ipynb


### Radial velocity analysis

In [4]:
rv_nb_fname = "08-radial_velocities.ipynb"
with open(rv_nb_fname) as f:
    nb = nbformat.read(f, as_version=4)

_ = ep.preprocess(nb, {'metadata': {'path': nb_dir}})
    
with open(nb_fname, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)
    
print("Finished running {:s}".format(rv_nb_fname))

Finished running 08-radial_velocities.ipynb


### Loop over the range of observation epochs, repeating the orbit and phase curve fit for each case.

In [5]:
ep = ExecutePreprocessor(timeout = 2400)

orbit_nb_fname = "09-orbit.ipynb"
phasecurve_nb_fname = "10-phase_curve.ipynb"

total_orbits = 40000
#total_orbits = 5000

for last_epoch in range(1, 6):
    with open(orbit_nb_fname) as f:
        nb = nbformat.read(f, as_version=4)
    
    orig_parameters = extract_parameters(nb)
    new_params = parameter_values(orig_parameters, last_epoch=last_epoch, total_orbits = total_orbits)
    new_nb = replace_definitions(nb, new_params, execute=False)

    _ = ep.preprocess(new_nb, {'metadata': {'path': nb_dir}})
    
    with open(orbit_nb_fname, 'w', encoding='utf-8') as f:
        nbformat.write(new_nb, f)
        
    print("Finished running {:s} with last epoch = {:d}".format(orbit_nb_fname, last_epoch))
    
    with open(phasecurve_nb_fname) as f:
        nb = nbformat.read(f, as_version=4)
    
    orig_parameters = extract_parameters(nb)
    new_params = parameter_values(orig_parameters, last_epoch=last_epoch)
    new_nb = replace_definitions(nb, new_params, execute=False)

    _ = ep.preprocess(new_nb, {'metadata': {'path': nb_dir}})
    
    with open(phasecurve_nb_fname, 'w', encoding='utf-8') as f:
        nbformat.write(new_nb, f)
        
    print("Finished running {:s} with last epoch = {:d}".format(phasecurve_nb_fname, last_epoch))

Finished running 09-orbit.ipynb with last epoch = 1
Finished running 10-phase_curve.ipynb with last epoch = 1
Finished running 09-orbit.ipynb with last epoch = 2
Finished running 10-phase_curve.ipynb with last epoch = 2
Finished running 09-orbit.ipynb with last epoch = 3
Finished running 10-phase_curve.ipynb with last epoch = 3
Finished running 09-orbit.ipynb with last epoch = 4
Finished running 10-phase_curve.ipynb with last epoch = 4
Finished running 09-orbit.ipynb with last epoch = 5
Finished running 10-phase_curve.ipynb with last epoch = 5
